# Random Linear Code Encription (RLCE) Python Implementation



Notes:
- It demonstrates the RLCE
  matrix pipeline (S, G1, A, P) and decryption algebraically.
- Decryption will only reliably succeed when encryption error weight = 0 (or when the error is within the decoding capability of a decoder you plug in).


In [8]:
import numpy as np
from typing import Tuple, Dict, List

In [7]:
# ----------------- GF(2) helpers -----------------
def gf2_mod(x):
    return np.asarray(x, dtype=np.uint8) & 1

def gf2_random(shape):
    return np.random.randint(0, 2, size=shape, dtype=np.uint8)

def gf2_eye(n):
    return np.eye(n, dtype=np.uint8)

def gf2_dot(A, B):
    # matrix multiplication mod 2
    return gf2_mod(np.dot(A.astype(np.uint8), B.astype(np.uint8)))

def gf2_row_swap(mat, i, j):
    mat[[i, j], :] = mat[[j, i], :]

def gf2_inverse(A: np.ndarray) -> np.ndarray:
    """
    Inverse of an n x n matrix over GF(2) using Gauss-Jordan elimination.
    Raises ValueError if singular.
    """
    A = np.array(A, dtype=np.uint8)
    n = A.shape[0]
    if A.shape[0] != A.shape[1]:
        raise ValueError("Matrix must be square to invert")
    aug = np.concatenate([A.copy(), gf2_eye(n)], axis=1)
    row = 0
    for col in range(n):
        pivot = None
        for r in range(row, n):
            if aug[r, col] == 1:
                pivot = r
                break
        if pivot is None:
            continue
        if pivot != row:
            gf2_row_swap(aug, pivot, row)
        # eliminate other rows
        for r in range(n):
            if r != row and aug[r, col] == 1:
                aug[r, :] ^= aug[row, :]
        row += 1
        if row == n:
            break
    left = aug[:, :n]
    if not np.array_equal(left, gf2_eye(n)):
        raise ValueError("Matrix is singular in GF(2) (no inverse)")
    inv = aug[:, n:]
    return gf2_mod(inv)

def gf2_solve(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    """
    Solve A x = b over GF(2). A: (m x n), b: (m,) or (m,1).
    Returns one solution x of length n. If no solution, raises ValueError.
    If multiple solutions exist, free variables are set to 0.
    """
    A = np.array(A, dtype=np.uint8)
    b = np.array(b, dtype=np.uint8).reshape(-1, 1)
    m, n = A.shape
    aug = np.concatenate([A.copy(), b.copy()], axis=1)
    row = 0
    pivot_cols = []
    for col in range(n):
        pivot = None
        for r in range(row, m):
            if aug[r, col] == 1:
                pivot = r
                break
        if pivot is None:
            continue
        if pivot != row:
            gf2_row_swap(aug, pivot, row)
        pivot_cols.append(col)
        for r in range(m):
            if r != row and aug[r, col] == 1:
                aug[r, :] ^= aug[row, :]
        row += 1
        if row == m:
            break
    # consistency check
    for r in range(row, m):
        if np.all(aug[r, :n] == 0) and aug[r, n] == 1:
            raise ValueError("No solution to linear system over GF(2)")
    x = np.zeros((n, 1), dtype=np.uint8)
    for r_idx, col in enumerate(pivot_cols):
        x[col, 0] = aug[r_idx, n]
    return x.reshape(-1)




In [4]:
def block_diag(blocks: List[np.ndarray]) -> np.ndarray:
    sizes = [b.shape[0] for b in blocks]
    total = sum(sizes)
    out = np.zeros((total, total), dtype=np.uint8)
    idx = 0
    for b in blocks:
        s = b.shape[0]
        out[idx:idx+s, idx:idx+s] = b
        idx += s
    return out

def permutation_matrix_from_perm(perm: np.ndarray) -> np.ndarray:
    n = len(perm)
    P = np.zeros((n, n), dtype=np.uint8)
    for i, j in enumerate(perm):
        P[i, j] = 1
    return P



In [5]:
# ----------------- RLCE GF(2) Implementation -----------------
class RLCE_GF2:
    def __init__(self, n: int, k: int, r: int = 1, seed: int = None):
        """
        n: number of underlying code columns (Gs has shape k x n)
        k: message length (rows of generator)
        r: number of random columns inserted after each real column
        """
        if seed is not None:
            np.random.seed(seed)
        self.n = n
        self.k = k
        self.r = r
        self.N = n * (r + 1)  # length after expansion
        self.Gs = gf2_random((k, n))  # underlying secret generator (toy)
        self.public_key = None
        self.private_key = None

    def _make_G1(self):
        # build G1 by interleaving real columns of Gs with r random columns each
        cols = []
        for j in range(self.n):
            cols.append(self.Gs[:, [j]])
            for _ in range(self.r):
                cols.append(gf2_random((self.k, 1)))
        G1 = np.hstack(cols)
        return gf2_mod(G1)

    def _random_invertible_block(self, size: int) -> np.ndarray:
        while True:
            M = gf2_random((size, size))
            try:
                _ = gf2_inverse(M)
                return M
            except ValueError:
                continue

    def keygen(self) -> Tuple[np.ndarray, Dict]:
        """
        Returns (public_key, private_key).
        public_key: k x N matrix over GF(2)
        private_key: dict containing S, Gs, A_blocks, perm
        """
        G1 = self._make_G1()  # k x N
        # block-diagonal A with invertible blocks (r+1 x r+1)
        blocks = [self._random_invertible_block(self.r + 1) for _ in range(self.n)]
        A = block_diag(blocks)
        S = self._random_invertible_block(self.k)
        perm = np.random.permutation(self.N)
        P = permutation_matrix_from_perm(perm)
        G_pub = gf2_dot(gf2_dot(gf2_dot(S, G1), A), P)
        self.public_key = G_pub
        self.private_key = {"S": S, "Gs": self.Gs, "A_blocks": blocks, "perm": perm, "P": P}
        return G_pub, self.private_key

    def encrypt(self, m_vec: np.ndarray, weight: int = 0) -> np.ndarray:
        """
        m_vec: 1 x k row vector (or length-k)
        weight: number of random error bits to flip (conceptual)
        """
        if self.public_key is None:
            raise ValueError("Public key not generated")
        m = m_vec.reshape(1, self.k).astype(np.uint8)
        y = gf2_dot(m, self.public_key)  # 1 x N
        if weight > 0:
            e = np.zeros((1, self.N), dtype=np.uint8)
            pos = np.random.choice(self.N, size=weight, replace=False)
            e[0, pos] = 1
            y ^= e
        return gf2_mod(y)

    def decrypt(self, y_vec: np.ndarray, priv: Dict) -> np.ndarray:
        """
        Decrypt ciphertext y_vec (1 x N) using priv key.
        Works for the no-error case (or if an external decoder is applied).
        """
        y = y_vec.reshape(1, self.N).astype(np.uint8)
        S = priv["S"]
        Gs = priv["Gs"]
        blocks = priv["A_blocks"]
        perm = priv["perm"]
        P = priv["P"]
        # Undo permutation: multiply by P.T
        y_unperm = gf2_dot(y, P.T)
        # Undo A blockwise
        Ainv_blocks = [gf2_inverse(b) for b in blocks]
        Ainv = block_diag(Ainv_blocks)
        tmp = gf2_dot(y_unperm, Ainv)  # 1 x N
        # collapse each (r+1) block by taking first element (the real column position)
        y_prime = []
        for i in range(self.n):
            block = tmp[0, i*(self.r+1):(i+1)*(self.r+1)]
            y_prime.append(block[0])
        y_prime = np.array(y_prime, dtype=np.uint8).reshape(1, self.n)
        # Solve Gs^T x^T = y_prime^T  => find x = m*S (1 x k)
        A_matrix = Gs.T  # shape (n x k)
        b_vector = y_prime.reshape(-1)
        x_solution = gf2_solve(A_matrix, b_vector)  # length k
        x_solution = x_solution.reshape(1, self.k)
        # invert S to recover m
        S_inv = gf2_inverse(S)
        m_recovered = gf2_dot(x_solution, S_inv)
        return m_recovered



In [6]:

np.random.seed(42)
n = 6
k = 3
r = 1
rlce = RLCE_GF2(n=n, k=k, r=r)
pub, priv = rlce.keygen()
print("Underlying Gs (k x n):\n", rlce.Gs)
m = gf2_random((1, k))
print("Plaintext m:", m)
y = rlce.encrypt(m, weight=0)
print("Ciphertext y (no error):", y)
m_out = rlce.decrypt(y, priv)
print("Decrypted m:", m_out)
assert np.array_equal(m.reshape(-1), m_out.reshape(-1))
print("OK: decryption succeeded (no-error case).")

Underlying Gs (k x n):
 [[0 0 1 1 1 1]
 [0 1 0 1 0 1]
 [0 1 1 0 0 0]]
Plaintext m: [[1 0 1]]
Ciphertext y (no error): [[1 1 0 0 1 1 1 1 0 1 1 0]]
Decrypted m: [[1 0 1]]
OK: decryption succeeded (no-error case).
